# NOTEBOOK PARA GENERAR EL DATAFRAME CON LA DATA DE TODOS LOS ARCHIVOS PARA AAFIT, BBFIT Y OTROS

<font size=5>La data para los hits la generaré con otro Notebook.</font>

Es importante mencionar que esta "limpieza" (generación del algoritmo) y obtención de data, me llevó aproximadamente 12 horas en general, desde el proceso creativo, pasando por la frustración, muchos intentos fallidos y hasta el momento de EUREKA!!!... XD.

Borré ya la mayoría de celdas que me llevaron al código de las siguientes dos celdas, el cual está mucho más pulido y limpio, pero creo que es importente mencionar por lo que pase y las ideas (algunas locas) que me surgieron para poder resolverlo.

Algunos ideas que tuve antes de llegar a decidir que no calcularia como tal el indice de aafit fueron:
* Solo quedarme con los archivos que tuvieran 100% data completa (terrible idea ya que son más del 50% los archivos que en algunos eventos - alrededor del 5% o menos - no tienen data de aafit).
* Verificar las longitudes de la cadena aafit y bbfit y comparar (idea bastante mala por cierto)
* Tratar de buscar por separado en otra celda solo la data de aafit y "pegarla" de alguna manera al dataframe final, hay que decir que esta idea también fué muy mala y no prosperó en lo absoluto (creo no tenia ni pies ni cabeza, momento de frustración máxima).

**NOTA:** Como siempre, la gran cuestión y prerrogativa humana es encontrar patrones en los datos y automatizar
búsquedas aún y cuando estos no se vean a simple vista o no sean fácilmente inferibles, tales como cuando 
el patrón es precisamente eso... que NO HAY PATRÓN!!!, esto aplica para el no indice ind_aafit

In [1]:
#Recordando algunas variables importantes, funciones y librerías que se requerirán
import os
import pandas as pd
import re
import math

#extractpath = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este path es para mi laptop en casa
extractpath = "F:\\DATA_TFM\\TXTs" #Este path es para mi PC en el trabajo
#extractpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este es para mi no tan nueva PC-Desktop

txtfilesarr = os.listdir(extractpath); txtfilesarr=sorted(txtfilesarr)
csv_path = "F:\\DATA_TFM\\CSVs\\"
#csv_path = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\"
#csv_path = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\"
csvarr = os.listdir(csv_path);csvarr=sorted(csvarr)

# Funcion para obtener los indices de un archivo
def indices(df):
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = [];#ind_aafit = []
    for i in range(data.shape[0]):
        tempstr = data[0][i]
        if "start_event" in tempstr: ind_startev.append(i)
        if "UTC" in tempstr: ind_runid.append(i)
        if "weights" in tempstr: ind_weights.append(i)
        if "muon" in tempstr: ind_muon.append(i)
        #if "aafit" in tempstr:  ##NO LO CALCULARE!!! OBTENDRE LOS VALORES DE OTRA FORMA
        #    ind_aafit.append(i) ##COMO TENGO EL INDICE DE BBFIT, RESTO 1 LINEA Y ASIGNO VALORES
        if "bbfit" in tempstr:   ##SEGUN LA EXISTENCIA O NO DE LA LINEA!!!! EUREKA!!!
            ind_bbfit.append(i)
        if "selected" in tempstr: ind_bb_selp.append(i)
        if "end_event" in tempstr: ind_endev.append(i)
    return(ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev)

#Función para verificar si los indices tienen el mismo # de elementos
def longitudes(i1,i2,i3,i4,i5,i6,i7):
    a=len(i1);b=len(i2);c=len(i3);d=len(i4);e=len(i5);f=len(i6);g=len(i7)
    #print(a,b,c,d,e,f,g)
    if (a==b and b==c and c==d and d==e and e==f and f==g): return(True)
    else: return(False)
    return()

### NOTA: Como siempre, la gran cuestión y prerrogativa humana es encontrar patrones en los datos y automatizar
### búsquedas aún y cuando estos no se vean a simple vista o no sean fácilmente inferibles, tales como cuando 
### el patrón es precisamente eso... que NO HAY PATRÓN!!!, esto aplica para el no indice ind_aafit

In [4]:
print(csvarr.index("MC_058730_numu_CC_a_reco.i3.csv"))
print(len(csvarr))
dif= 1838 - 432
print(dif)
print(csvarr[432+1405])
print(1406+432)

432
1838
1406
1838


In [ ]:
############################################################################################################
###### ESTE CODIGO GENERARÁ UN DATAFRAME CON LA DATA DE BBFIT, AAFIT Y OTRAS DE ""TODOS"" LOS ARCHIVOS #####
############################################################################################################

#Se requiere un dataframe donde iremos guardando cada fila de data obtenida de cada archivo
columnas = ['runID','frameID','trigger_counter','interactionID','aafit_azimut',
            'aafit_zenit','aafit_lambda','aafit_beta','bbfit_azimut','bbfit_zenit','bbfit_quality',
            'muon_azimut','muon_zenit','muon_energy'] #La data del muon la añadi el 30 Jun, lo habia "olvidado" XD
df_data1 = pd.DataFrame(columns=columnas)

consecutivo=0 #variable auxiliar para guardar el indice del ultimo elemento del archivo

inicio = 432 #Es el indice del archivo donde debe empezar porque ya la PC de casa esta trabajando en los otros

# Bucle para recorrer TODOS los archivos y obtener la data y metadata de TODOS
for k in range(1406):
    #print("Ciclo", k)
    file = csv_path+csvarr[k+inicio]
    print("El archivo trabajando es: ",file)
    data = pd.read_csv(file, header=None) # Dataframe del archivo en tratamiento
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = []

    #Generacion de indices para cada archivo
    ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

    #Como el runid es identico para todos los datos en un mismo archivo, uso el primero que aparece en el dataframe
    runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])
    #Igual como el interactionID es el mismo para todos, asigno 1 si el archivo es "numu" y 2 si es "anumu"
    interactionID = [lambda:1, lambda:2]["anumu" in str(file)]()
    # Bucle para recorrer el archivo en curso y obtener la data relevante
    print("Entrando al bucle para meter renglones al dataframe") 
    for i in range(len(ind_startev)):
        #Codigo para obtener los demas datos por cada evento y despues asignarlo a los hits
        #Divido la cadena en los valores que la componen para después extraer frameid y trigger_counter
        temprunid = list(data[0][ind_runid[i]].split(" "))
        tempmuon = list(data[0][ind_muon[i]].split(" "))
        tempbbfit = list(data[0][ind_bbfit[i]].split(" "))
        #Extrayendo las variables frameid, trigger_counter de este renglon---> esta data si cambia ojo!!
        frameID = temprunid[1]; trigger_counter = temprunid [2]
        #Obteniendo y generando Datos para aafit
        #Para calcular la data solo haremos la diferencia y si exista data la calculamos si no
        #pues ponemos valores fuera de rango para saber después que no hubo data como -3pi
        laafit = data[0][ind_bbfit[i]-1]
        if "aafit" in laafit:
            tempaafit = list(data[0][ind_bbfit[i]-1].split(" "))
            aafit_azimut = math.atan2(float(tempaafit[3]),float(tempaafit[2]))
            aafit_zenit = math.acos(float(tempaafit[4]))
            aafit_lambda = tempaafit[-2]
            aafit_beta = tempaafit [-1]
        else:
            aafit_azimut=-3*math.pi;aafit_zenit = -3*math.pi;aafit_lambda=1;aafit_beta=-1
        #Obteniendo y generando Datos para bbfit
        X = float(tempbbfit[2]); Y = float(tempbbfit[3]); Z = float(tempbbfit[4])
        #Hay que verificar si X o Y son nan y asignar el valor de bbfit_azimut
        if math.isnan(X):   bbfit_azimut = -3*math.pi
        elif math.isnan(Y): bbfit_azimut = -3*math.pi
        else: bbfit_azimut = math.atan2(Y,X)
        bbfit_zenit = math.acos(Z)
        bbfit_quality = float(tempbbfit[-1]) 
        #Obteniendo y generando la data del muon
        X = float(tempmuon[1]); Y = float(tempmuon[2]); Z = float(tempmuon[3])
        muon_azimut = math.atan2(Y,X)
        muon_zenit = math.acos(Z)
        muon_energy = float(tempmuon[-2])
        #Añadimos la data obtenida y la generada al dataframe
        renglon = [runID, frameID, trigger_counter, interactionID,aafit_azimut, aafit_zenit,
                   aafit_lambda,aafit_beta, bbfit_azimut,bbfit_zenit,bbfit_quality,muon_azimut,muon_zenit,muon_energy]
        #Añadiendo datos al dataframe.
        df_data1.loc[i+consecutivo] = renglon
    consecutivo = df_data1.shape[0]

print("Fin de la extracción de la data")

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_058730_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_058770_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_058770_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_058790_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_058790_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_058810_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_058810_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_058830_anumu_CC_a_reco.i3.csv
Entrando al bucle para m

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_060120_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_060120_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_060130_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_060130_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_060150_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_060310_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_060320_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_060320_numu_CC_a_reco.i3.csv
Entrando al bucle para m

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_061210_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_061390_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_061410_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_061410_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_061440_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_061440_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_061450_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_061450_numu_CC_a_reco.i3.csv
Entrando al bucle para m

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_062560_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_062730_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_062730_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_062780_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_062780_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_062800_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_062800_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_062930_anumu_CC_a_reco.i3.csv
Entrando al bucle para 

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_065050_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_065050_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_065100_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_065100_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_065120_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_065120_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_065140_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_065140_numu_CC_a_reco.i3.csv
Entrando al bucle para m

...

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_069390_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_069430_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_069440_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_069440_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_069470_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_069470_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_069480_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_069480_numu_CC_a_reco.i3.csv
Entrando al bucle para me

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_071760_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_071760_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_071780_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_071780_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_071810_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_071810_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_072100_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_072100_numu_CC_a_reco.i3.csv
Entrando al bucle para m

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_074240_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_074480_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_074480_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_074550_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_074550_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_074590_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_074590_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_074610_anumu_CC_a_reco.i3.csv
Entrando al bucle para 

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_076790_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_076810_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_076810_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_076930_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_076930_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_077070_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_077070_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_077110_anumu_CC_a_reco.i3.csv
Entrando al bucle para m

El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_078610_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_078620_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_078620_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_078670_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_078670_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_078850_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_078850_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  F:\DATA_TFM\CSVs\MC_078950_anumu_CC_a_reco.i3.csv
Entrando al bucle para m

In [ ]:
### PRUEBAS... BORRAR ESTA CELDA AL FINAL

#Viendo como quedó el dataframe
print("El número de filas del dataframe es:", df_data1.shape[0])

print("Una muestra de las filas y la data en ellas")

df_data1.tail(10)

#df_data1.iloc[0:250]

In [ ]:
## Una vez que tenemos el dataframe con toda la data relevante de aafit y bbfit
## debemos guardar esta data a disco

#outputpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CLEANDATA\\"
outputpath = "F:\DATA_TFM\CLEANDATA\\"
    
df_data1.to_csv (outputpath+"data_aafit_bbfit_muon"+".csv", index = None)#, header=None)

#Nota: Empezó a las 10:18pm del 30 Jun para 1136 archivos